In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import pandas as pd
import random

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
pd.set_option('display.expand_frame_repr', False)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
df = pd.read_csv("movies_metadata.csv",  skiprows = lambda i: i % 4)
df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
1,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,45325,tt0112302,en,Tom and Huck,"A mischievous young boy, Tom Sawyer, witnesses...",2.561161,/sGO5Qa55p7wTu7FJcX4H4xIVKvS.jpg,"[{'name': 'Walt Disney Pictures', 'id': 2}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,97.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,The Original Bad Boys.,Tom and Huck,False,5.4,45.0
2,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 27, 'nam...",NaN,12110,tt0112896,en,Dracula: Dead and Loving It,When a lawyer shows up at the vampire's doorst...,5.430331,/xve4cgfYItnOhtzLYoTwTVy5FGr.jpg,"[{'name': 'Columbia Pictures', 'id': 5}, {'nam...","[{'iso_3166_1': 'FR', 'name': 'France'}, {'iso...",1995-12-22,0.0,88.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,Dracula: Dead and Loving It,False,5.7,210.0
3,False,NaN,52000000,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",NaN,524,tt0112641,en,Casino,The life of the gambling paradise – Las Vegas ...,10.137389,/xo517ibXBDdYQY81j0WIG7BVcWq.jpg,"[{'name': 'Universal Pictures', 'id': 33}, {'n...","[{'iso_3166_1': 'FR', 'name': 'France'}, {'iso...",1995-11-22,116112375.0,178.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,No one stays at the top forever.,Casino,False,7.8,1343.0
4,False,NaN,60000000,"[{'id': 28, 'name': 'Action'}, {'id': 35, 'nam...",NaN,11517,tt0113845,en,Money Train,A vengeful New York transit cop decides to ste...,7.337906,/jSozzzVOR2kfXgTUuGnbgG2yRFi.jpg,"[{'name': 'Columbia Pictures', 'id': 5}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-11-21,35431113.0,103.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"Get on, or GET OUT THE WAY!",Money Train,False,5.4,224.0


In [ ]:
df.shape

(11366, 24)

In [ ]:
print(df.isnull().sum())

adult                        0
belongs_to_collection    10258
budget                       0
genres                       0
homepage                  9382
id                           0
imdb_id                      3
original_language            3
original_title               0
overview                   238
popularity                   1
poster_path                 88
production_companies         0
production_countries         0
release_date                20
revenue                      2
runtime                     68
spoken_languages             2
status                      21
tagline                   6241
title                        2
video                        2
vote_average                 2
vote_count                   2
dtype: int64


In [ ]:
df.isnull().sum()

adult                        0
belongs_to_collection    10258
budget                       0
genres                       0
homepage                  9382
id                           0
imdb_id                      3
original_language            3
original_title               0
overview                   238
popularity                   1
poster_path                 88
production_companies         0
production_countries         0
release_date                20
revenue                      2
runtime                     68
spoken_languages             2
status                      21
tagline                   6241
title                        2
video                        2
vote_average                 2
vote_count                   2
dtype: int64

In [ ]:
M = 2500
C = df['vote_average'].mean()

def weighted_rating(r, v, M, C):
    return (v / (v + M) * r) + (M / (v + M) * C)

df["weighted_rating"] = weighted_rating(df["vote_average"],
                                            df["vote_count"], M, C)

df.sort_values("weighted_rating", ascending=False).head(10)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,weighted_rating
710,False,NaN,63000000,"[{'id': 18, 'name': 'Drama'}]",http://www.foxmovies.com/movies/fight-club,550,tt0137523,en,Fight Club,A ticking-time-bomb insomniac and a slippery s...,63.869599,/adw6Lq9FiC9zjYEpOqfq03ituwp.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'DE', 'name': 'Germany'}, {'is...",1999-10-15,100853753.0,139.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Mischief. Mayhem. Soap.,Fight Club,False,8.3,9678.0,7.746323
5719,False,NaN,165000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 18, '...",http://www.interstellarmovie.net/,157336,tt0816692,en,Interstellar,Interstellar chronicles the adventures of a gr...,32.213481,/nBNZadXqJSdt05SHLqgT0HuC5Gm.jpg,"[{'name': 'Paramount Pictures', 'id': 4}, {'na...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",2014-11-05,675120017.0,169.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Mankind was born on Earth. It was never meant ...,Interstellar,False,8.1,11187.0,7.643898
87,False,NaN,55000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,13,tt0109830,en,Forrest Gump,A man with a low IQ has accomplished great thi...,48.307194,/yE5d3BUhE8hCnkMUJOo1QDoOGNz.jpg,"[{'name': 'Paramount Pictures', 'id': 4}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1994-07-06,677945399.0,142.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"The world will never be the same, once you've ...",Forrest Gump,False,8.2,8147.0,7.590187
1215,False,"{'id': 119, 'name': 'The Lord of the Rings Col...",93000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",http://www.lordoftherings.net/,120,tt0120737,en,The Lord of the Rings: The Fellowship of the Ring,"Young hobbit Frodo Baggins, after inheriting a...",32.070725,/bxVxZb5O9OxCO0oRUNdCnpy9NST.jpg,"[{'name': 'WingNut Films', 'id': 11}, {'name':...","[{'iso_3166_1': 'NZ', 'name': 'New Zealand'}, ...",2001-12-18,871368364.0,178.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,One ring to rule them all,The Lord of the Rings: The Fellowship of the Ring,False,8.0,8892.0,7.473958
5012,False,NaN,100000000,"[{'id': 18, 'name': 'Drama'}, {'id': 37, 'name...",http://unchainedmovie.com/,68718,tt1853728,en,Django Unchained,"With the help of a German bounty hunter, a fre...",19.785025,/5WJnxuw41sddupf8cwOxYftuvJG.jpg,"[{'name': 'Columbia Pictures', 'id': 5}, {'nam...","[{'iso_3166_1': 'US', 'name': 'United States o...",2012-12-25,425368238.0,165.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,"Life, liberty and the pursuit of vengeance.",Django Unchained,False,7.8,10297.0,7.370784
5532,False,NaN,100000000,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...",http://www.thewolfofwallstreet.com/,106646,tt0993846,en,The Wolf of Wall Street,A New York stockbroker refuses to cooperate in...,16.382422,/vK1o5rZGqxyovfIhZyMELhk03wO.jpg,"[{'name': 'Paramount Pictures', 'id': 4}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",2013-12-25,392000694.0,180.0,"[{'iso_639_1': 'fr', 'name': 'Français'}, {'is...",Released,EARN. SPEND. PARTY.,The Wolf of Wall Street,False,7.9,6768.0,7.280376
7578,False,NaN,175000000,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...",http://movies.disney.com/inside-out,150540,tt2096673,en,Inside Out,"Growing up can be a bumpy road, and it's no ex...",23.985587,/aAmfIX3TT40zUHGcCKrlOZRKC7u.jpg,"[{'name': 'Walt Disney Pictures', 'id': 2}, {'...","[{'iso_3166_1': 'US', 'name': 'United States o...",2015-06-09,857611174.0,94.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Meet the little voices inside your head.,Inside Out,False,7.9,6737.0,7.278297
89,False,"{'id': 94032, 'name': 'The Lion King Collectio...",45000000,"[{'id': 10751, 'name': 'Family'}, {'id': 16, '...",http://movies.

In [ ]:
df["overview"].head()

0    Cheated on, mistreated and stepped on, the wom...
1    A mischievous young boy, Tom Sawyer, witnesses...
2    When a lawyer shows up at the vampire's doorst...
3    The life of the gambling paradise – Las Vegas ...
4    A vengeful New York transit cop decides to ste...
Name: overview, dtype: object

In [ ]:
df = df[~df["title"].duplicated(keep='last')]
df = df[~df["title"].isna()]
df = df[~df["overview"].isna()]

In [ ]:
tfidf = TfidfVectorizer(stop_words="english")


In [ ]:
tfidf_matrix = tfidf.fit_transform(df['overview'])


In [ ]:
tfidf_matrix = tfidf_matrix.astype(np.float32)

In [ ]:
tfidf_matrix.shape


(10882, 37304)

In [ ]:
tfidf_matrix

<10882x37304 sparse matrix of type '<class 'numpy.float32'>'
	with 298586 stored elements in Compressed Sparse Row format>

In [ ]:
df['title'].shape

(10882,)

In [ ]:
tfidf.get_feature_names_out()

array(['00', '000', '000th', ..., '찾기', '첫사랑', 'ﬁve'], dtype=object)

In [ ]:
cosine_sim = cosine_similarity(tfidf_matrix,
                               tfidf_matrix)
cosine_sim.shape


(10882, 10882)

In [ ]:
cosine_sim[1]

array([0.00922137, 0.99999994, 0.        , ..., 0.        , 0.01560123,
       0.        ], dtype=float32)

In [ ]:
indices = pd.Series(df.index, index=df['title'])

In [ ]:
indices.index.value_counts().sort_values(ascending=False)


Waiting to Exhale                   1
Twelve Monkeys                      1
Vampire in Brooklyn                 1
Black Sheep                         1
Last Summer in the Hamptons         1
                                   ..
The Man with the Rubber Head        1
The Untameable Whiskers             1
The Fat and Lean Wrestling Match    1
St. Michael Had a Rooster           1
Betrayal                            1
Name: title, Length: 10882, dtype: int64

In [ ]:
indices["Cinderella"]

8898

In [ ]:
indices["Django Unchained"]

5012

In [ ]:
# Check if the movie title "Cinderella" is present in the index
if "Cinderella" in indices:
    movie_index = indices["Cinderella"]
    cosine_sim_cinderella = cosine_sim[movie_index]
    print(cosine_sim_cinderella)
else:
    print("Movie title 'Cinderella' is not found in the DataFrame index.")

[0. 0. 0. ... 0. 0. 0.]


In [ ]:
cosine_sim[movie_index]


array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [ ]:
similarity_scores = pd.DataFrame(cosine_sim[movie_index],
                                 columns=["score"])

In [ ]:
movie_indices = similarity_scores.sort_values("score", ascending=False)[1:11].index


In [ ]:
df['title'].iloc[movie_indices]

9100                  Die Konferenz der Tiere
4653                         Little Criminals
208                        Maybe... Maybe Not
2073    The Golem: How He Came Into the World
4586                            The Way South
883           Pandora and the Flying Dutchman
1633                          Death in Venice
4105        Harlan: In the Shadow of Jew Süss
9254                            Island Guests
6033                      The Obama Deception
Name: title, dtype: object

In [ ]:
def content_based_recommender(title, cosine_sim, dataframe):
    # index'leri olusturma
    indices = pd.Series(dataframe.index, index=dataframe['title'])
    indices = indices[~indices.index.duplicated(keep='last')]
    # title'ın index'ini yakalama
    movie_index = indices[title]
    # title'a gore benzerlik skorlarını hesapalama
    similarity_scores = pd.DataFrame(cosine_sim[movie_index], columns=["score"])
    # kendisi haric ilk 10 filmi getirme
    movie_indices = similarity_scores.sort_values("score", ascending=False)[1:11].index
    return dataframe['title'].iloc[movie_indices]

In [ ]:
content_based_recommender("Django Unchained", cosine_sim, df)


3700    The First Day of the Rest of Your Life
8091                      Hollywood Vice Squad
7435                 Cocaine Cowboys: Reloaded
6949                                    Amador
6692                                     Tempo
7427        Chelsea Peretti: One of the Greats
4210                                    DeVour
4293                          Tomorrow We Move
101                       The Age of Innocence
4678                                     Miral
Name: title, dtype: object

In [ ]:
content_based_recommender("Cinderella", cosine_sim, df)


9100                  Die Konferenz der Tiere
4653                         Little Criminals
208                        Maybe... Maybe Not
2073    The Golem: How He Came Into the World
4586                            The Way South
883           Pandora and the Flying Dutchman
1633                          Death in Venice
4105        Harlan: In the Shadow of Jew Süss
9254                            Island Guests
6033                      The Obama Deception
Name: title, dtype: object

In [ ]:
content_based_recommender("Interstellar", cosine_sim, df)


1930                Saved!
6984        Saturday Night
591           Dry Cleaning
4901       West of Memphis
3112     Diary of the Dead
10859               Origin
1688            Storyville
2556               Stealth
9839         Hide and Seek
11333    American Violence
Name: title, dtype: object

In [ ]:
content_based_recommender('Fight Club', cosine_sim, df)


6584                        Merlin's Apprentice
8252                           The Night Before
7902                                After Death
5839                 X-Men: Days of Future Past
10754                   Santo en Anónimo mortal
8130                                Salem Falls
9598     The Holy Modal Rounders: Bound to Lose
10796                              White Coffin
6912                             American Heist
3832                Bless the Beasts & Children
Name: title, dtype: object

In [ ]:
def calculate_cosine_sim(dataframe):
    tfidf = TfidfVectorizer(stop_words='english')
    dataframe['overview'] = dataframe['overview'].fillna('')
    tfidf_matrix = tfidf.fit_transform(dataframe['overview'])
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    return cosine_sim

In [ ]:
cosine_sim = calculate_cosine_sim(df)
content_based_recommender('Fight Club', cosine_sim, df)

6584                        Merlin's Apprentice
8252                           The Night Before
7902                                After Death
5839                 X-Men: Days of Future Past
10754                   Santo en Anónimo mortal
8130                                Salem Falls
9598     The Holy Modal Rounders: Bound to Lose
10796                              White Coffin
6912                             American Heist
3832                Bless the Beasts & Children
Name: title, dtype: object

In [ ]:
def content_based_recommender_user_input(cosine_sim, dataframe):

    user_input_movie = input("What movie do you want to watch next? Enter the movie title: ")

    if user_input_movie not in dataframe['title'].values:
        print("Movie not found in the dataset. Please try another movie.")
        return

    user_input_rating = input("Enter your desired minimum rating score (0-10): ")

    try:
        user_input_rating = float(user_input_rating)
        if user_input_rating < 0 or user_input_rating > 10:
            print("Invalid rating score. Please enter a number between 0 and 10.")
            return
    except ValueError:
        print("Invalid rating score. Please enter a valid number between 0 and 10.")
        return

    recommendations = content_based_recommender(user_input_movie, cosine_sim, dataframe, user_input_rating)

    print("\nRecommended movies based on similarity to '{}' with a rating score of {} or above:".format(user_input_movie, user_input_rating))
    print("--------------------------------------------------------------------------------------------")
    for index, movie in recommendations.iterrows():
        print("Title:", movie['title'])
        print("Overview:", movie['overview'])
        genres = ', '.join([genre['name'] for genre in eval(movie['genres'])])
        print("Genres:", genres)
        print("Original Language:", movie['original_language'])
        print("Popularity:", movie['popularity'])
        print("Runtime:", movie['runtime'])
        print("Vote Average:", movie['vote_average'])
        print("Vote Count:", movie['vote_count'])
        print("--------------------------------------------------------------------------------------------")


def content_based_recommender(title, cosine_sim, dataframe, min_rating):
    indices = pd.Series(dataframe.index, index=dataframe['title'])
    indices = indices[~indices.index.duplicated(keep='last')]
    movie_index = indices[title]
    similarity_scores = pd.DataFrame(cosine_sim[movie_index], columns=["score"])
    movie_indices = similarity_scores[(similarity_scores['score'] > 0) &
                                       (dataframe['vote_average'] >= min_rating)].sort_values("score", ascending=False)[1:11].index
    return dataframe.loc[movie_indices]

content_based_recommender_user_input(cosine_sim, df)


What movie do you want to watch next? Enter the movie title: Django Unchained
Enter your desired minimum rating score (0-10): 8.5

Recommended movies based on similarity to 'Django Unchained' with a rating score of 8.5 or above:
--------------------------------------------------------------------------------------------
Title: My Foolish Heart
Overview: After a long absence, Mary Jane visits her schoolfriend Eloise, and Eloise's daughter Ramona. Eloise drinks too much and is unhappily married to Lew Wengler. Eloise falls asleep and remembers her time with her true love, Walt Dreiser, at the beginning of the Second World War. She recalls the events that lead up to her split with Mary Jane, and how Lew married Eloise rather than Mary Jane.
Genres: 
Original Language: en
Popularity: 0.065284
Runtime: 98.0
Vote Average: 10.0
Vote Count: 1.0
--------------------------------------------------------------------------------------------
Title: Recollections of the Yellow House
Overview: In his 

<ipython-input-148-5c8506a5ba51>:53: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  movie_indices = similarity_scores[(similarity_scores['score'] > 0) &
